E2 Novice

Brian Bannon

In [11]:
import numpy as np
import pandas as pd

In [12]:
def calculate_single_moment(bridge_length: float, load: float, load_position: float, x: float) -> float:
    """
    Calculates the internal bending moment at a specific point along a simply supported bridge due to a single point load.
    :param bridge_length: Total length of the bridge in meters.
    :param load: Magnitude of the applied point load in kilonewtons (kN).
    :param load_position: Distance from the left support to the applied load in meters.
    :param x: Position along the bridge where the bending moment is calculated in meters.
    :return: Bending moment at position x in kilonewton-meters (kN-m).
    """
    if x <= load_position:
        b = bridge_length - load_position
        return (load * b * x) / bridge_length
    else:
        return (load * load_position * (bridge_length - x)) / bridge_length


In [13]:
def calculate_deflection(bridge_length: float, load: float, load_position: float, x: float, ei: float) -> float:
    """
    Calculates the vertical deflection (sagging) at a specific point along a simply supported bridge due to a single point load.
    :param bridge_length: Total span of the bridge in meters.
    :param load: Force applied in kilonewtons (kN).
    :param load_position: Distance from the left support to the applied load in meters.
    :param x: Position along the bridge to calculate deflection in meters.
    :param ei: Flexural rigidity (stiffness) in kN-m² (product of Elastic Modulus and Moment of Inertia).
    :return: Deflection at position x in meters (float, positive = downward).
    """
    l = bridge_length
    p = load
    a = load_position
    b = l - a

    if x <= a:
        deflection = (p * b * x) / (6 * l * ei) * (l ** 2 - b ** 2 - x ** 2)
    else:
        deflection = (p * a) / (6 * l * ei) * ((l - x) * (2 * l * x - x ** 2 - a ** 2))

    return deflection

In [14]:
def create_analysis_summary(bridge_specs: dict, load_specs: dict) -> dict:
    """
    Creates a summary of the structural analysis for a simply supported bridge under a single point load.
    :param bridge_specs:
    :param load_specs:
    :return:
    """
    l = bridge_specs['length']
    ei = bridge_specs['EI']

    p = load_specs['magnitude']
    a = load_specs['position']

    test_points = np.linspace(0, l, 101)
    moments = [calculate_single_moment(l, p, a, x) for x in test_points]
    max_moment = max(moments)
    moment_index = moments.index(max_moment)
    max_moment_location = test_points[moment_index]

    deflections = [calculate_deflection(l, p, a, x, ei) for x in test_points]
    max_deflection = max(deflections)
    deflection_index = deflections.index(max_deflection)
    max_deflection_location = test_points[deflection_index]

    if max_deflection > 0:
        deflection_ratio = l / max_deflection
    else:
        deflection_ratio = float('inf')

    status = 'PASS' if max_deflection <= l / 360 else 'FAIL'

    return {
        'max_moment': max_moment,
        'max_moment_location': max_moment_location,
        'max_deflection': max_deflection,
        'max_deflection_location': max_deflection_location,
        'deflection_ratio': deflection_ratio,
        'status': status
    }

In [ ]:

frame = pd.DataFrame({
    'Name': 'Pedestrian Footbridge in City Park',
    'Length': 10,

})

